In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F

import lime
from lime.lime_tabular import RecurrentTabularExplainer
import glog
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
sys.path.append("../")
import xfinai_config
from data_layer.base_dataset import FuturesDatasetRecurrent
from utils import base_io
%matplotlib inline

## 0 Load Model

In [2]:
future_index = 'IC'
from model_layer.model_hub import LSTM
params = base_io.load_best_params(future_index, LSTM.name)
train_data, val_data, test_data = base_io.load_data('IC')
train_dataset, val_dataset, test_dataset = base_io.get_dataset('IC', params=params)
train_x = np.array(train_dataset.data_x)
test_x =  np.array(test_dataset.data_x)
train_y = np.array(train_dataset.data_y)
test_y =  np.array(test_dataset.data_y)

In [4]:
model_path = '../model_layer/trained_models/IC/LSTM.pth'
model = LSTM(
   params
)
model.load_state_dict(torch.load(model_path))

KeyError: 'input_size'

In [ ]:
def pred_func(batch_x):
    pre_result = np.array([])
    full_batch_num = batch_x.shape[0] // params['batch_size']
    full_batch_length = full_batch_num * params['batch_size']
    batch_x = batch_x[:full_batch_length]
    
    for batch_num in range(full_batch_num):
        current_batch = batch_x[batch_num*params['batch_size'] : (batch_num+1)*params['batch_size']]
        pred =  model(torch.tensor(current_batch).float().detach()).detach().numpy()
        pre_result = np.append(pre_result, pred)
    
    return pre_result

In [ ]:
explainer = RecurrentTabularExplainer(train_x, training_labels=train_y, feature_names=train_dataset.features_list,discretize_continuous=False, mode='regression')

In [ ]:
test_y.max()

In [ ]:
rand_instance = np.random.randint(0, len(test_x))
exp = explainer.explain_instance(test_x[784], pred_func, num_features=10, num_samples=6400)

In [ ]:
exp.show_in_notebook()

In [17]:
exp.save_to_file('./Lime_explaination.html')

In [ ]:
plt.figure(figsize=[10,8], dpi=100)
exp.as_pyplot_figure()